### Imports 👽

In [1]:
import pandas as pd
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_classes.scenario import ScenarioSettings
from data_classes.enums import Scenario
import sampling
from experiment_functions import get_experiment_function

### Setup Scenario 🧪

In [2]:
SCENARIO = Scenario.SUM_SINES
settings = ScenarioSettings(SCENARIO)
input_dim = settings.INPUT_DIM
test_function = get_experiment_function(SCENARIO)

### Generate Data 📊

In [3]:
sobol_train_x = sampling.generate_sobol_sequence(
    num_samples = 8192,
    dim = input_dim
)

sobol_train_y = test_function(sobol_train_x)

mc_train_x = sampling.generate_random_sequence(
    num_samples = 2**13,
    dim = input_dim
)
mc_train_y = test_function(mc_train_x)

In [4]:
sobol_train_together = np.hstack((sobol_train_x, sobol_train_y.reshape(-1, 1)))
sobol_train_df = pd.DataFrame(
    data=sobol_train_together,
    columns=settings.COL_NAMES
)

mc_train_together = np.hstack((mc_train_x, mc_train_y.reshape(-1, 1)))
mc_train_df = pd.DataFrame(
    data=mc_train_together,
    columns=settings.COL_NAMES
)

### Store Data 💿

In [5]:
output_dir = os.path.abspath(os.path.join('..', 'data', SCENARIO.value, 'input'))
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'sobol_sample.csv')
sobol_train_df.to_csv(output_path, index=False)

output_path = os.path.join(output_dir, 'mc_sample.csv')
mc_train_df.to_csv(output_path, index=False)